- 데이터 보간 후 사용한 결과임
- ads_id, mda_id 추가
- 시차피처 추가

In [45]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [46]:
ads_year = pd.read_csv("아이브1년치_참여데이터.csv")
ads_list = pd.read_csv("df_list.csv")


/var/folders/hl/90p2p86n5cs6t3p_1xwxdx2r0000gn/T/ipykernel_85676/1970805062.py:2: DtypeWarning: Columns (12,13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  ads_list = pd.read_csv("df_list.csv")


In [47]:
ads_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6953146 entries, 0 to 6953145
Data columns (total 11 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   Unnamed: 0      int64 
 1   rpt_time_date   object
 2   rpt_time_time   int64 
 3   ads_idx         int64 
 4   mda_idx         int64 
 5   rpt_time_clk    int64 
 6   rpt_time_turn   int64 
 7   rpt_time_scost  int64 
 8   rpt_time_acost  int64 
 9   rpt_time_cost   int64 
 10  rpt_time_earn   int64 
dtypes: int64(10), object(1)
memory usage: 583.5+ MB


In [48]:
# 광고목록에서 현재 진행중인 광고 찾기

import datetime

ads_list['ads_edate'] = pd.to_datetime(ads_list['ads_edate'], errors='coerce')

# 2025. 8. 29 전 날짜 삭제
compare_date = datetime.date(2025, 8, 29)
condition1 = ads_list['ads_edate'].dt.date >= compare_date
ads_list_filtered = ads_list[condition1]

# delyn이 N인 행만 남기기
ads_list_filtered = ads_list_filtered[ads_list_filtered['delyn'] == 'N']


ads_list_c = ads_list_filtered[['ads_idx', 'ads_sdate','ads_edate']]

# ads_list_c에 있는 ads_idx만 필터링
df = ads_year[ads_year['ads_idx'].isin(ads_list_c['ads_idx'])]

# 두 데이터 합치기 (ads_idx 기준)
# merged_df = pd.merge(df_filtered, ads_list_c, on='ads_idx', how='inner')

/var/folders/hl/90p2p86n5cs6t3p_1xwxdx2r0000gn/T/ipykernel_85676/556888358.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ads_list['ads_edate'] = pd.to_datetime(ads_list['ads_edate'], errors='coerce')


In [49]:
# 클릭수보다 전환수가 큰 경우 전환수를 클릭수로 맞춤
df.loc[
    df['rpt_time_turn'] > df['rpt_time_clk'], 
    'rpt_time_turn'
] = df['rpt_time_clk']

# 확인
over_conversion_after = df[df['rpt_time_turn'] > df['rpt_time_clk']]
print("클릭수보다 전환수가 큰 행 수 (조정 후):", len(over_conversion_after))


클릭수보다 전환수가 큰 행 수 (조정 후): 0


In [50]:
# 고유 ID 생성: ads_idx와 mda_idx를 결합
df["unique_id"] = (
    df["ads_idx"].astype(str).str.zfill(6) +   # ads_idx는 항상 6자리
    df["mda_idx"].astype(str).str.zfill(4)     # mda_idx는 항상 4자리
)

# 필요하다면 숫자로 변환
df["unique_id"] = df["unique_id"].astype(int)

/var/folders/hl/90p2p86n5cs6t3p_1xwxdx2r0000gn/T/ipykernel_85676/3277818209.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["unique_id"] = (
/var/folders/hl/90p2p86n5cs6t3p_1xwxdx2r0000gn/T/ipykernel_85676/3277818209.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["unique_id"] = df["unique_id"].astype(int)


In [51]:
# rpt_time_date를 datetime으로 변환
df["rpt_time_date"] = pd.to_datetime(df["rpt_time_date"])

# 그룹화하면서 ads_idx, mda_idx도 첫 번째 값으로 가져오기
df_day = (
    df.groupby(["unique_id", "rpt_time_date"], as_index=False)
    .agg({
        "rpt_time_turn": "sum",
        "ads_idx": "first",
        "mda_idx": "first"
    })
    .rename(columns={"rpt_time_turn": "turn_sum"})
)


/var/folders/hl/90p2p86n5cs6t3p_1xwxdx2r0000gn/T/ipykernel_85676/32143721.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rpt_time_date"] = pd.to_datetime(df["rpt_time_date"])


In [52]:
import pandas as pd

# 8/27, 8/28, 8/29 날짜 지정
target_dates = pd.to_datetime(['2025-08-27','2025-08-28','2025-08-29'])

# 해당 날짜 중 하나라도 있는 광고 ID
ads_with_recent = df_day[df_day['rpt_time_date'].isin(target_dates)]['unique_id'].unique()

# 해당 광고만 남기기
df_day = df_day[df_day['unique_id'].isin(ads_with_recent)]

print("8/27, 8/28, 8/29 중 하나라도 데이터가 있는 광고 수:", df_day['unique_id'].nunique())
print(df_day.head())


8/27, 8/28, 8/29 중 하나라도 데이터가 있는 광고 수: 1281
      unique_id rpt_time_date  turn_sum  ads_idx  mda_idx
9113   99820824    2024-08-25         0     9982      824
9114   99820824    2024-08-28        10     9982      824
9115   99820824    2024-08-29        62     9982      824
9116   99820824    2024-08-30        87     9982      824
9117   99820824    2024-08-31        85     9982      824


In [53]:
# 광고별 데이터 날짜 길이
ad_days = df_day.groupby('unique_id')['rpt_time_date'].nunique()

# 통계량 확인
stats = ad_days.describe()  # count, mean, std, min, 25%, 50%, 75%, max 포함
print(stats)

count    1281.000000
mean       98.362217
std       110.650922
min         1.000000
25%        18.000000
50%        49.000000
75%       140.000000
max       399.000000
Name: rpt_time_date, dtype: float64


In [54]:
# 날짜가 30일 이상인 것만 가져가기 정규분포에 의해서
date_counts = df_day.groupby("unique_id")["rpt_time_date"].nunique()
valid_ids = date_counts[date_counts >= 30].index
df_day = df_day[df_day["unique_id"].isin(valid_ids)]

In [55]:
#### 확인용임!!!!!
# 기준 날짜
end_date = pd.to_datetime("2025-08-29")

# id별 빈 날짜 개수 계산
missing_days = (
    df_day.groupby("unique_id")
    .agg(
        start_date=("rpt_time_date", "min"),
        observed_days=("rpt_time_date", "nunique")
    )
    .reset_index()
)

# 전체 구간 길이 = (마지막날짜 - 시작날짜 + 1일)
missing_days["total_days"] = (end_date - missing_days["start_date"]).dt.days + 1

# 빈 날짜 개수
missing_days["missing_days"] = missing_days["total_days"] - missing_days["observed_days"]

print(missing_days.head())


   unique_id start_date  observed_days  total_days  missing_days
0   99820824 2024-08-25            195         370           175
1   99820845 2024-09-01            139         363           224
2   99820850 2025-03-18             89         165            76
3   99820851 2025-03-18             88         165            77
4   99820990 2025-03-04             52         179           127


In [56]:
# 데이터 보간 작업 시작.
# 오늘 날짜
end_date = pd.to_datetime("2025-08-29")

filled_list = []

for uid, group in df_day.groupby("unique_id"):
    # 아이디별 시작 날짜 ~ 8/29까지 생성
    start_date = group['rpt_time_date'].min()
    all_dates = pd.date_range(start=start_date, end=end_date, freq="D")

    # 날짜 기준으로 reindex
    g = group.set_index("rpt_time_date").reindex(all_dates)

    # flag: 기존 데이터 있으면 1, 없으면 0
    g["flag"] = g["turn_sum"].notna().astype(int)

    # 원래 값 없던 날은 turn_sum = 0
    g["turn_sum"] = g["turn_sum"].fillna(0)

    # ID 다시 넣기
    g["unique_id"] = uid
    g = g.reset_index().rename(columns={"index": "rpt_time_date"})

    filled_list.append(g)

df_fill = pd.concat(filled_list, ignore_index=True)

print(df_fill.head(15))


   rpt_time_date  unique_id  turn_sum  ads_idx  mda_idx  flag
0     2024-08-25   99820824       0.0   9982.0    824.0     1
1     2024-08-26   99820824       0.0      NaN      NaN     0
2     2024-08-27   99820824       0.0      NaN      NaN     0
3     2024-08-28   99820824      10.0   9982.0    824.0     1
4     2024-08-29   99820824      62.0   9982.0    824.0     1
5     2024-08-30   99820824      87.0   9982.0    824.0     1
6     2024-08-31   99820824      85.0   9982.0    824.0     1
7     2024-09-01   99820824      43.0   9982.0    824.0     1
8     2024-09-02   99820824      42.0   9982.0    824.0     1
9     2024-09-03   99820824      36.0   9982.0    824.0     1
10    2024-09-04   99820824      61.0   9982.0    824.0     1
11    2024-09-05   99820824      32.0   9982.0    824.0     1
12    2024-09-06   99820824      44.0   9982.0    824.0     1
13    2024-09-07   99820824      35.0   9982.0    824.0     1
14    2024-09-08   99820824      70.0   9982.0    824.0     1


In [57]:
df_fill.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187687 entries, 0 to 187686
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   rpt_time_date  187687 non-null  datetime64[ns]
 1   unique_id      187687 non-null  int64         
 2   turn_sum       187687 non-null  float64       
 3   ads_idx        119111 non-null  float64       
 4   mda_idx        119111 non-null  float64       
 5   flag           187687 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 8.6 MB


In [58]:
# unique_id 문자열 변환
df_fill["unique_id"] = df_fill["unique_id"].astype(str)

# ads_idx가 NaN인 경우에만 채우기
df_fill.loc[df_fill["ads_idx"].isna(), "ads_idx"] = (
    df_fill.loc[df_fill["ads_idx"].isna(), "unique_id"].str[:-4].astype(float)
)

# mda_idx가 NaN인 경우에만 채우기
df_fill.loc[df_fill["mda_idx"].isna(), "mda_idx"] = (
    df_fill.loc[df_fill["mda_idx"].isna(), "unique_id"].str[-4:].astype(float)
)


In [59]:
df_fill.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187687 entries, 0 to 187686
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   rpt_time_date  187687 non-null  datetime64[ns]
 1   unique_id      187687 non-null  object        
 2   turn_sum       187687 non-null  float64       
 3   ads_idx        187687 non-null  float64       
 4   mda_idx        187687 non-null  float64       
 5   flag           187687 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 8.6+ MB


In [60]:
df_fill['year'] = df_fill['rpt_time_date'].dt.year
df_fill['month'] = df_fill['rpt_time_date'].dt.month
df_fill['day'] = df_fill['rpt_time_date'].dt.day
df_fill["unique_id"] = df_fill["unique_id"].astype(int)

In [61]:
df_fill["turn_sum_log"] = np.log1p(df_fill["turn_sum"])

In [62]:
# category 타입으로 변환
df_fill["unique_id"] = df_fill["unique_id"].astype("category")
df_fill["ads_idx"] = df_fill["ads_idx"].astype("category")
df_fill["mda_idx"] = df_fill["mda_idx"].astype("category")

In [63]:
# 시차피처 하나 생성 ver.
# 그룹별 lag feature 생성 (예: 1일, 2일 전 값)
df_fill["lag1"] = df_fill.groupby("unique_id")["turn_sum_log"].shift(1)
df_fill["lag2"] = df_fill.groupby("unique_id")["turn_sum_log"].shift(2)
df_fill["lag3"] = df_fill.groupby("unique_id")["turn_sum_log"].shift(3)

In [64]:
df_fill.head()

,rpt_time_date,unique_id,turn_sum,ads_idx,mda_idx,flag,year,month,day,turn_sum_log,lag1,lag2,lag3
0,2024-08-25,99820824,0.0,9982.0,824.0,1,2024,8,25,0.000000,NaN,NaN,NaN
1,2024-08-26,99820824,0.0,9982.0,824.0,0,2024,8,26,0.000000,0.000000,NaN,NaN
2,2024-08-27,99820824,0.0,9982.0,824.0,0,2024,8,27,0.000000,0.000000,0.0,NaN
3,2024-08-28,99820824,10.0,9982.0,824.0,1,2024,8,28,2.397895,0.000000,0.0,0.0
4,2024-08-29,99820824,62.0,9982.0,824.0,1,2024,8,29,4.143135,2.397895,0.0,0.0


In [65]:
df_fill = df_fill.dropna()

In [66]:
df_fill.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185347 entries, 3 to 187686
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   rpt_time_date  185347 non-null  datetime64[ns]
 1   unique_id      185347 non-null  category      
 2   turn_sum       185347 non-null  float64       
 3   ads_idx        185347 non-null  category      
 4   mda_idx        185347 non-null  category      
 5   flag           185347 non-null  int64         
 6   year           185347 non-null  int32         
 7   month          185347 non-null  int32         
 8   day            185347 non-null  int32         
 9   turn_sum_log   185347 non-null  float64       
 10  lag1           185347 non-null  float64       
 11  lag2           185347 non-null  float64       
 12  lag3           185347 non-null  float64       
dtypes: category(3), datetime64[ns](1), float64(5), int32(3), int64(1)
memory usage: 14.4 MB


## 데이터 분리

In [67]:
train_list = []
valid_list = []
test_list = []

for ad_id, group in df_fill.groupby("unique_id"):
    group = group.sort_values("rpt_time_date")
    
    # 마지막 1일 예측 → test, 그 전날 1일 → valid, 나머지 → train
    test = group.tail(1)
    valid = group.iloc[-2:-1]
    train = group.iloc[:-1]
    
    train_list.append(train)
    valid_list.append(valid)
    test_list.append(test)

train_df = pd.concat(train_list)
valid_df = pd.concat(valid_list)
test_df = pd.concat(test_list)

print("Train shape:", train_df.shape)
print("Valid shape:", valid_df.shape)
print("Test shape:", test_df.shape)


Train shape: (184567, 13)
Valid shape: (780, 13)
Test shape: (780, 13)


In [68]:
features = ['year','month','day','unique_id','ads_idx','mda_idx','flag','lag1', 'lag2', 'lag3']  # 필요하면 요일/월 등 추가 가능
target = "turn_sum_log"

## 평가함수

In [69]:
# 로그변환된 target을 지수변환하기 위한 함수
def exp_transform(y):
    return np.expm1(y)  # log1p -> expm1, 0 값도 안전하게 처리

# MAPE 계산 함수
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.clip(y_true, 1e-8, None))) * 100

## 모델

### 랜덤포레스트

In [70]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np


# 모델 초기화
rf = RandomForestRegressor(random_state=42)

# 1️⃣ Train 데이터 학습
rf.fit(train_df[features], train_df[target])

# 2️⃣ Train 데이터 예측
pred_val = rf.predict(valid_df[features])

pred_val_exp = exp_transform(pred_val)
y_train_exp = exp_transform(valid_df[target])

# 3️⃣ Train 성능 평가
mse_train = mean_squared_error(y_train_exp, pred_val_exp)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train_exp, pred_val_exp)
r2_train = r2_score(y_train_exp, pred_val_exp)
mape_train = mean_absolute_percentage_error(y_train_exp, pred_val_exp)

print("valid 성능:")
print(f"R2: {r2_train:.4f}, MAE: {mae_train:.2f}, MSE: {mse_train:.2f}, RMSE: {rmse_train:.2f}, MAPE: {mape_train:.2f}%")



valid 성능:
R2: 0.7984, MAE: 1.22, MSE: 37.47, RMSE: 6.12, MAPE: 245300940.58%


In [71]:
# 4️⃣ Test 데이터 예측
pred_test = rf.predict(test_df[features])

pred_test_exp = exp_transform(pred_test)
y_test_exp = exp_transform(test_df[target])

# 5️⃣ Test 성능 평가
mse_test = mean_squared_error(y_test_exp, pred_test_exp)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test_exp, pred_test_exp)
r2_test = r2_score(y_test_exp, pred_test_exp)
mape_test = mean_absolute_percentage_error(y_test_exp, pred_test_exp)

print("Test 성능:")
print(f"R2: {r2_test:.4f}, MAE: {mae_test:.2f}, MSE: {mse_test:.2f}, RMSE: {rmse_test:.2f}, MAPE: {mape_test:.2f}%")

Test 성능:
R2: 0.7327, MAE: 2.38, MSE: 73.80, RMSE: 8.59, MAPE: 1193139863.07%


### XGBoost

In [72]:
import xgboost as xgb

# 모델 초기화
xgb_model = xgb.XGBRegressor(enable_categorical=True, random_state=42)

# 1️⃣ Train 데이터 학습
xgb_model.fit(train_df[features], train_df[target])

# 2️⃣ Train 데이터 예측
pred_val = xgb_model.predict(valid_df[features])

pred_val_exp = exp_transform(pred_val)
y_train_exp = exp_transform(valid_df[target])

# 3️⃣ Train 성능 평가
mse_train = mean_squared_error(y_train_exp, pred_val_exp)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train_exp, pred_val_exp)
r2_train = r2_score(y_train_exp, pred_val_exp)
mape_train = mean_absolute_percentage_error(y_train_exp, pred_val_exp)

print("valid 성능:")
print(f"R2: {r2_train:.4f}, MAE: {mae_train:.2f}, MSE: {mse_train:.2f}, RMSE: {rmse_train:.2f}, MAPE: {mape_train:.2f}%")



valid 성능:
R2: 0.7739, MAE: 1.86, MSE: 42.02, RMSE: 6.48, MAPE: 693778241.77%


In [73]:
# 4️⃣ Test 데이터 예측
pred_test = xgb_model.predict(test_df[features])

pred_test_exp = exp_transform(pred_test)
y_test_exp = exp_transform(test_df[target])

# 5️⃣ Test 성능 평가
mse_test = mean_squared_error(y_test_exp, pred_test_exp)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test_exp, pred_test_exp)
r2_test = r2_score(y_test_exp, pred_test_exp)
mape_test = mean_absolute_percentage_error(y_test_exp, pred_test_exp)

print("Test 성능:")
print(f"R2: {r2_test:.4f}, MAE: {mae_test:.2f}, MSE: {mse_test:.2f}, RMSE: {rmse_test:.2f}, MAPE: {mape_test:.2f}%")

Test 성능:
R2: 0.6557, MAE: 2.65, MSE: 95.07, RMSE: 9.75, MAPE: 1207169964.32%


### lgbm

In [74]:
import lightgbm as lgb

lgb_model = lgb.LGBMRegressor(random_state=42)


# 1️⃣ Train 데이터 학습
lgb_model.fit(train_df[features], train_df[target])

# 2️⃣ Train 데이터 예측
pred_val = lgb_model.predict(valid_df[features])

pred_val_exp = exp_transform(pred_val)
y_train_exp = exp_transform(valid_df[target])

# 3️⃣ Train 성능 평가
mse_train = mean_squared_error(y_train_exp, pred_val_exp)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train_exp, pred_val_exp)
r2_train = r2_score(y_train_exp, pred_val_exp)
mape_train = mean_absolute_percentage_error(y_train_exp, pred_val_exp)

print("valid 성능:")
print(f"R2: {r2_train:.4f}, MAE: {mae_train:.2f}, MSE: {mse_train:.2f}, RMSE: {rmse_train:.2f}, MAPE: {mape_train:.2f}%")



[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006302 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1944
[LightGBM] [Info] Number of data points in the train set: 184567, number of used features: 10
[LightGBM] [Info] Start training from score 0.469184
valid 성능:
R2: 0.3735, MAE: 2.52, MSE: 116.43, RMSE: 10.79, MAPE: 893584847.07%


In [75]:
# 4️⃣ Test 데이터 예측
pred_test = lgb_model.predict(test_df[features])

pred_test_exp = exp_transform(pred_test)
y_test_exp = exp_transform(test_df[target])

# 5️⃣ Test 성능 평가
mse_test = mean_squared_error(y_test_exp, pred_test_exp)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test_exp, pred_test_exp)
r2_test = r2_score(y_test_exp, pred_test_exp)
mape_test = mean_absolute_percentage_error(y_test_exp, pred_test_exp)

print("Test 성능:")
print(f"R2: {r2_test:.4f}, MAE: {mae_test:.2f}, MSE: {mse_test:.2f}, RMSE: {rmse_test:.2f}, MAPE: {mape_test:.2f}%")

Test 성능:
R2: 0.6927, MAE: 2.52, MSE: 84.86, RMSE: 9.21, MAPE: 1096300020.31%
